In [ ]:
import sys
import os
import time

import numpy as np
import scipy.ndimage
import pandas as pd
import matplotlib.pyplot as plt

from osgeo import gdal
from osgeo import osr
from osgeo.gdalconst import *
driver = gdal.GetDriverByName('GTiff')
driver.Register()



In [ ]:
def createRaster(filename, array, coords_system, resol, origin, nodata=-9999, dtype=GDT_Float32):
    print(filename)
    rows, cols = array.shape
    pixelWidth, pixelHeight = resol, resol
    originX = origin[0]
    originY = origin[1]
    
    ds = driver.Create(filename, cols, rows, 1, GDT_Float32)
    ds.SetGeoTransform((originX, pixelWidth, 0, originY, 0, -pixelHeight))
    band = ds.GetRasterBand(1)
    band.WriteArray(array)
    band.FlushCache()
    band.SetNoDataValue(-9999)
    ds.SetProjection(coords_system.ExportToWkt())
    
    ds = None

In [ ]:
def setCellsCoords(rows, cols):
    coords = np.empty((rows,cols),dtype=str)
    for i in range(rows):
        for j in range(cols):
            coords[i][j] = str(int(i))+'.'+str(int(j))
    return coords



def prepareOutputFile(infile, coords):
    rows, cols = coords.shape
    infile.write("Timestep\t")
    for i in range(rows):
        for j in range(cols):
            infile.write(coords[i][j])
            infile.write("\t")
    infile.write("\n")
    return infile



def writeLineOutputFile(infile,timestep,values):
    rows, cols = values.shape
    infile.write("{}\t".format(timestep))
    for i in range(rows):
        for j in range(cols):
            infile.write("{0:.3f}\t".format(values[i][j]))
    infile.write("\n")
    return infile



def updateFront(previous_contiguous_saturated, contiguous_saturated, previous_front, d, current_time):
    frontwise_previous_contiguous_saturated = np.where(d>0.0, previous_contiguous_saturated, -200)
    frontwise_contiguous_saturated = np.where(d>0.0, contiguous_saturated, -200)
    satur_to_nonsatur_only = np.where(frontwise_contiguous_saturated != frontwise_previous_contiguous_saturated, 1, None)
    #print(satur_to_nonsatur_only)
    satur_to_nonsatur_for_front = np.where(frontwise_contiguous_saturated != frontwise_previous_contiguous_saturated, 200, frontwise_contiguous_saturated)
    satur_to_nonsatur_only_rows, satur_to_nonsatur_only_cols = satur_to_nonsatur_only.shape
    #print(satur_to_nonsatur_only.shape, satur_to_nonsatur_for_front.shape)
    #print()
    current_front = previous_front
    for i in range(satur_to_nonsatur_only_rows):
        for j in range(satur_to_nonsatur_only_cols):
            if satur_to_nonsatur_for_front[i][j] != 200:
                continue

            neighborhood = []

            if j==0:
                west = None
            elif frontwise_contiguous_saturated[i][j-1] == -200:
                west = None
            else:
                west = satur_to_nonsatur_for_front[i][j-1]
            neighborhood.append(west)

            if i==0:
                north = None
            elif frontwise_contiguous_saturated[i-1][j] == -200:
                north = None
            else:
                north = satur_to_nonsatur_for_front[i-1][j]
            neighborhood.append(north)

            try:
                east = satur_to_nonsatur_for_front[i][j+1]
                if frontwise_contiguous_saturated[i][j+1] == -200:
                    east = None
            except:
                east = None  
            neighborhood.append(east)

            try:
                south = satur_to_nonsatur_for_front[i+1][j]
                if frontwise_contiguous_saturated[i+1][j] == -200:
                    south = None
            except:
                south = None
            neighborhood.append(south)

            #print(i, j, neighborhood)
            for neighbor in neighborhood:
                if neighbor == 0:
                    current_front -= 1
                if neighbor == 1:
                    current_front += 1
    #print(current_time, current_front)
    return current_front



def calculate_Qin_D8(Qin_array, Qout_array, direction):
    #calculates Qin in each cell based on receiving Qout from neighboring cell(s) 
    
    Qin_array += np.where(np.roll(direction, (-1,1), (1,0)) == 1,
                                      np.roll(Qout_array, (-1,1), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (0,1), (1,0)) == 2,
                                      np.roll(Qout_array, (0,1), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (1,1), (1,0)) == 3,
                                      np.roll(Qout_array, (1,1), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (-1,0), (1,0)) == 4,
                                      np.roll(Qout_array, (-1,0), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (0,0), (1,0)) == 5,
                                      0,
                                      0) #5 is the outlet, must force discharge out
    Qin_array += np.where(np.roll(direction, (1,0), (1,0)) == 6,
                                      np.roll(Qout_array, (1,0), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (-1,-1), (1,0)) == 7,
                                      np.roll(Qout_array, (-1,-1), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (0,-1), (1,0)) == 8,
                                      np.roll(Qout_array, (0,-1), (1,0)),
                                      0)
    Qin_array += np.where(np.roll(direction, (1,-1), (1,0)) == 9,
                                      np.roll(Qout_array, (1,-1), (1,0)),
                                      0)

In [ ]:
def drainTheWatershed(watershed_cells, elevationValues, tanbetaValues, lddValues, resol, structure, run_foldername,
                       originX, originY, outlet_cell, gauges_cells,
                       Area_ratio_list, normalized_Vsoilwater_list, saturFront_list,
                       waterSoilStorageValues, wTableHeightValues, 
                       total_area, saturatedWatershedSoilWaterVolume,
                       D, fi, Sy, Ks, m, psi_b, n_days, deltat=1, 
                       decaying_transmissivity=True):
    '''
    This function simulates the draining of the aquifer from a condition of complete saturation of the whole watershed
    The purpose is to related ratio of saturated areas over total area AND water content in the soil of the watershed.
    Both quantities are calculated and recorded at each time step.
        
    ### Input parameters ###
    watershed_cells: array of cells that are part of the watershed (excludes all the NA, null value cells)
    elevationValues: array of elevation values of cells, i.e. DEM values
    tanbetaValues: array of local slope of each cell
    lddValues: flow direction according to D8
    gauge_cells: list of cells where values are printed out (x,y as in coordinate system)
    Area_ratio_list: saturated area over total area at each time step
    normalized_Voilwater_list: water in the soil (storage) at each time step, normalized by saturatedWatershedSoilWaterVolume    
    waterSoilStorageValues: array of volumetric water content of cells, in saturated conditions
    wTableHeightValues: array of thickness of groundwater table, measured from the bottom (bedrock/impervious)
    total_area: area of the watershed
    saturatedWatershedSoilWaterVolume: max possible water volume when all cells in the watershed are saturated
    D: thickness of aquifer in saturated condition; i.e. distance of ground surface from bottom (bedrock/impervious)
    fi: porosity of soil
    Sy: specific yield, proportion of water that flows laterally after groundwater table lowers of a quantity deltad
    Ks: hydraulic conductivity in saturated conditions
    m: coefficient of transmissivity decay
    psi_b: bubbling pressure (0.1 meters)
    n_days: number of days of simulation
    deltat: length of time step, in hours
    decaying transmissivity: True if assumption that transimissivity decays exponentially with saturated layer thickness
    '''
    
    rows, cols = elevationValues.shape
    initial_storage = waterSoilStorageValues
    stopping_condition = False
    

    n_steps = 24 * n_days
    if n_days <= 30:
        n_steps = n_days*24
    elif n_days <= 90:
        n_steps = 30*24 + (n_days-30)*4
    elif n_days <= 180:
        n_steps = 30*24 + 60*4 + (n_days-90)*2
    elif n_days <= 360:
        n_steps = 30*24 + 60*4 + 90*2 + (n_days-180)*1
    else:
        n_steps = 30*24 + 60*4 + 90*2 + 180*1 + (n_days-360)//7 + (n_days-360)%7
    printout_steps = np.geomspace(1, n_steps+1, 20, dtype=int)

    
    Qindict = {}
    Qoutdict = {}
    Qdict={}
    Sdict = {}
    Edict = {}
    ddict = {}
    runoffVoldict = {}
    
    current_time = 0
    previous_outlet_satur_index = 0
    previous_contiguous_saturated = np.where(watershed_cells==1, 1, watershed_cells)
    previous_front = 0
    
    Qindict[current_time] = np.zeros((rows, cols))
    Qoutdict[current_time] = np.zeros((rows, cols))
    Sdict[current_time] = waterSoilStorageValues
    Edict[current_time] = elevationValues
    ddict[current_time] = wTableHeightValues
    runoffVoldict[current_time] = np.zeros((rows, cols))
    
    current_time += deltat
    while current_time < n_steps*deltat and stopping_condition == False:
        print("current time step: {}".format(current_time))
        Qindict[current_time] = np.zeros((rows, cols))
        Qoutdict[current_time] = np.zeros((rows, cols))
        Sdict[current_time] = np.zeros((rows, cols))
        Edict[current_time] = np.zeros((rows, cols))
        ddict[current_time] = np.zeros((rows, cols))
        runoffVoldict[current_time] = np.zeros((rows, cols))
        
        previous_d = ddict[current_time-deltat]
        T = Ks * previous_d
        if decaying_transmissivity == True:
            T = np.where(previous_d < (D - psi_b),
                         (Ks*D) * np.exp(-1*(D-psi_b-previous_d)*(Sy)/ m),
                         T)

        Qout = T * resol * tanbetaValues
        Qoutdict[current_time] = Qout
        direction = lddValues
        calculate_Qin_D8(Qindict[current_time], Qoutdict[current_time], direction)
        Qin = Qindict[current_time]

        #calculate current storage. Storage cannot be more than saturated condition and cannot be < 0
        deltaS = (Qin - Qout) * deltat
        #no_deltaS = np.where(deltaS == 0, 1, 0)
        previous_S = Sdict[current_time - deltat]
        S = previous_S + deltaS
        runoffVol = np.where(np.greater(S, initial_storage),
                            (S - initial_storage)/deltat,
                            0)
        S = np.where(S > initial_storage,
                     initial_storage, 
                     S)
        S = np.where(S < 0, #previous_S + deltaS < 0,
                     0,
                     S)
        Sdict[current_time] = S
        
        #calculate current thickness of saturated layer. Cannot be more than D=dmax and cannot be < 0
        deltad = deltaS / (Sy*resol*resol)
        d = previous_d + deltad
        previous_d = ddict[current_time-deltat]
        d = np.where(d > D,
                     D, 
                     d)
        d = np.where(d < 0, #previous_d + deltad < 0,
                     0, 
                     d)
        ddict[current_time] = d
        
        
        #saturated area contributing to runoff to outlet
        saturated_cells = np.where(d>0.0, 0, d)
        saturated_cells = np.where(d >= (D-psi_b), 1, saturated_cells)
        contiguous_saturated = scipy.ndimage.label(saturated_cells, structure)[0]
        if current_time in [-1]:#10,20,50,100,150,200,250,300]:
            createRaster(run_foldername+watershed_nick+"_S_{}.tif".format(current_time),
                         S, coords_system, resol, [originX, originY])
            createRaster(run_foldername+watershed_nick+"_d_{}.tif".format(current_time),
                         d, coords_system, resol, [originX, originY])
            createRaster(run_foldername + watershed_nick + "_{}_saturated_cells.tif".format(current_time), 
                         saturated_cells, coords_system, resol, [originX, originY])
            
        
        outlet_satur_index = contiguous_saturated[outlet_cell[0]][outlet_cell[1]]
        #print(outlet_satur_index, outlet_cell[0], outlet_cell[1])
        if outlet_satur_index > 0:
            contiguous_saturated = np.where(d>0.0, contiguous_saturated, 0)
            contiguous_saturated = np.where(contiguous_saturated == outlet_satur_index, 1, 0)
            if current_time in printout_steps:#10,20,50,100,150,200,250,300]:
                createRaster(run_foldername + watershed_nick + "_{}_contiguous_saturated.tif".format(current_time), 
                             contiguous_saturated, coords_system, resol, [originX, originY], nodata=0)
        else:
            print("\n\nWARNING: the outlet turned unsaturated at time step {}.".format(current_time))
            contiguous_saturated = np.where(d>0.0, contiguous_saturated, 0)
            contiguous_saturated = np.where(contiguous_saturated > outlet_satur_index, 1, 0)
            createRaster(working_dir + watershed_nick + "_{}_end_contiguous_saturated.tif".format(current_time), 
                         contiguous_saturated, coords_system, resol, [originX, originY], nodata=0)
            stopping_condition = True
        
        
        #calculate saturated area in watershed, cells where groundwater table not below bubbling pressure distance 
        #from ground surface
        saturated_area = np.sum(contiguous_saturated) * resol * resol
        #print(current_time, saturated_area)
        area_ratio = saturated_area/total_area
        Area_ratio_list.append(area_ratio)
        
        # calculate sum of water content in soil for all cells, saturated and non-saturated
        watershed_soilStorage = np.where(watershed_cells==1, S, 0)
        watershedSoilWaterVolume = np.sum(watershed_soilStorage)
        normalized_Vsoilwater_list.append(watershedSoilWaterVolume/saturatedWatershedSoilWaterVolume)
        watershed_soilStorage = np.where(watershed_cells==1, S, None)
        
        #calculate front of saturated area
        #current_front = updateFront(previous_contiguous_saturated, contiguous_saturated, previous_front, d, current_time)
        #saturated_front = 0
        #saturated_front += np.sum(contiguous_saturated[:,1:] != contiguous_saturated[:,:-1])
        #saturated_front += np.sum(contiguous_saturated[1:,:] != contiguous_saturated[:-1,:])
        #saturated_front += np.sum(contiguous_saturated[0,:] == 1) + np.sum(contiguous_saturated[-1,:] == 1)
        #saturated_front += np.sum(contiguous_saturated[:,0] == 1) + np.sum(contiguous_saturated[:,-1] == 1)
        #saturFront_list.append(current_front*resol)
        #previous_contiguous_saturated = contiguous_saturated
        #previous_front = current_front


        #for gauge in gauges_cells:
            #print("At {} water table height (d) is : {}".format(gauge, ddict[current_time][gauge[0]][gauge[1]]))
            #print("At {} water elevation is : {} m asl".format(gauge, Edict[current_time][gauge[0]][gauge[1]]))
            #print("At {} water storage (S) is : {}".format(gauge, Sdict[current_time][gauge[0]][gauge[1]]))
            #print("At {} runoff is {} m^3\n".format(gauge, runoffVoldict[current_time][gauge[0]][gauge[1]]))
        
        
        Qindict[current_time-deltat] = None
        Qoutdict[current_time-deltat] = None
        ddict[current_time-deltat] = None
        Sdict[current_time-deltat] = None
        Edict[current_time-deltat] = None
        runoffVoldict[current_time-deltat] = None
        
        if current_time in [-665,-666]:
            #filename = working_dir+watershed_nick+"_saturated_{}.tif".format(current_time)
            
            createRaster(run_foldername+watershed_nick+"_Qin_{}.tif".format(current_time),
                         Qin, coords_system, resol, [originX, originY])
            createRaster(run_foldername+watershed_nick+"_Qout_{}.tif".format(current_time),
                         Qout, coords_system, resol, [originX, originY])
            createRaster(run_foldername+watershed_nick+"_contiguous_saturated_{}.tif".format(current_time),
                         contiguous_saturated, coords_system, resol, [originX, originY], nodata=0)
            
        
        if current_time > 30*24:
            deltat = 6
        elif current_time > 30*24+60*4:
            deltat = 12
        elif current_time > 30*24+60*4+90*12:
            deltat = 24
        elif current_time > 30*24+60*4+90*12+180*1:
            deltat = 24*7
        
        current_time += deltat
    
    createRaster(run_foldername+watershed_nick+"_saturated.tif", 
                     saturated_cells, coords_system, resol, [originX, originY])
    createRaster(run_foldername+watershed_nick+"_d_end.tif", 
                     d, coords_system, resol, [originX, originY])
    createRaster(run_foldername+watershed_nick+"_S_end.tif", 
                     S, coords_system, resol, [originX, originY])
    
    for i in range(saturated_cells.shape[0]):
        for j in range(saturated_cells.shape[1]):
            if saturated_cells[i][j] == 1:
                x = originX + resol*j
                y = originY - resol*i
                #print(i,j,x,y)


    
    
def print_Aratio_SoilWatVol_relationship(filename, Area_ratio_list,normalized_Vsoilwater_list,saturFront_list):
    '''
    The function prints out the curve (corresponding values) to a txt file (filename).
    The curve can thus be reused later to estimate saturated excess contributing areas given 
    a certain amount of water in the soil
    '''
    relationship_df = pd.DataFrame({"AreaRatio": Area_ratio_list,
                                  "norm_SoilWatVol": normalized_Vsoilwater_list})
                                    #,
                                   #"Saturated_Front": saturFront_list})
    relationship_df.to_csv(filename, sep='\t')


In [ ]:
def plotCurve(normalized_Vsoilwater_list, Area_ratio_list, saturFront_list,
              run_folder, watershed_name, watershed_nick,
              decaying_transmissivity, m):
    #plot the curve (saturated area ratio vs normalized soil water content) and save as jpg

    fig,ax = plt.subplots(figsize=(10,5.5))
    ax.set_title(watershed_name + ' - Saturated area ratio vs Soil water content', fontsize=20)
    ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_xlabel("Normalized soil water content", fontsize=16, labelpad=0, rotation='horizontal')
    ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=13)

    spaced_xticks = [min(normalized_Vsoilwater_list),
                     min(normalized_Vsoilwater_list)+(max(normalized_Vsoilwater_list)-min(normalized_Vsoilwater_list))*0.25,
                     min(normalized_Vsoilwater_list)+(max(normalized_Vsoilwater_list)-min(normalized_Vsoilwater_list))*0.5,
                     min(normalized_Vsoilwater_list)+(max(normalized_Vsoilwater_list)-min(normalized_Vsoilwater_list))*0.75,
                     max(normalized_Vsoilwater_list)]
    #spaced_xticks = [0.0, 0.25, 0.5, 0.75, 1.0]
    print(spaced_xticks)
    ax.set_xticks(spaced_xticks)
    ax.set_xticklabels(['{0:.2f}'.format(spaced_xticks[0])+"\nNo area saturated",
                        '{0:.2f}'.format(spaced_xticks[1]),
                        '{0:.2f}'.format(spaced_xticks[2]),
                        '{0:.2f}'.format(spaced_xticks[3]),
                        '{0:.2f}'.format(spaced_xticks[-1])+"\nAll areas saturated"], 
                       fontsize=13)

    ax.plot(normalized_Vsoilwater_list, Area_ratio_list, color='k', linewidth=3)

    if decaying_transmissivity == False:
        plt.gcf().text(0.15, 0.8, "No transmissivity decay", 
                       fontsize=14, color='blue', fontweight="bold")
        fig.show()
        plt.savefig(run_folder + watershed_nick + "_satAreaRatio_SoilWaterVol_noDecay.jpg".format(m), dpi=85)
    else:
        plt.gcf().text(0.15, 0.8, "decay transmissivity\nparameter m = {}".format(m), 
                       fontsize=14, color='blue', fontweight="bold")
        fig.show()
        plt.savefig(run_folder + watershed_nick + "_satAreaRatio_SoilWaterVol_Decay_{}.jpg".format(m), dpi=85)

        

In [ ]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
watershed_name = "Hubbard #7"
watershed_foldername = "hubbard_brook"
watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep +
               watershed_nick + os.path.sep)
dem = working_dir + watershed_nick + "_10dem.tif"
tanbeta = working_dir + watershed_nick + "_tanbeta.tif"
ldd = working_dir + watershed_nick + "_ldd.tif"

################################### soil parameters ##############################################################
D = 2  #thickness of unconfined aquifer, shallow groundwater

#parameters for sandy loam  
Ks = 20  #micrometers per second
Ks = Ks * 3600 / 1000000  #meters per hour
decaying_transmissivity = True
m = 0.1  #this should be big number, so the exponent go to 0 (really? How big?)
psi_b = 0.1  #bubbling pressure / capillary head, for sand
fi = 0.45  #porosity
Sy = 0.24  #specific yield

################################# simulation run parameters ######################################################
n_days = 20000  #length of simulation
deltat = 1  #timestep
run_number = 1
outlet_coords = [759560, 4868910] #array of points, one point (x,y) for each row, where to record values
gauges_coords = [[759560, 4868910]] #array of points, one point (x,y) for each row, where to record values
#gauges_coords = [[277914.8, 4867532.8]] #array of points, one point (x,y) for each row, where to record values


elevations_ds = gdal.Open(dem, GA_ReadOnly)
band = elevations_ds.GetRasterBand(1)
cols = elevations_ds.RasterXSize
rows = elevations_ds.RasterYSize
print(rows, cols)
elevationValues = band.ReadAsArray(0, 0, cols, rows)
nodata = band.GetNoDataValue()
elevationValues = np.ma.masked_equal(elevationValues, nodata)
geotransform = elevations_ds.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
raster_origin = [originX, originY]
resol = np.round(geotransform[1])
print(originX, originY, resol)
#coords_system = osr.SpatialReference()
#coords_system.ImportFromEPSG(26919)
prj = elevations_ds.GetProjection()
coords_system = osr.SpatialReference(wkt=prj)
#proj_coords_system_string = srs.GetAttrValue('projcs')
#geog_coords_system_string = srs.GetAttrValue('geogcs')

tanbeta_ds = gdal.Open(tanbeta, GA_ReadOnly)
band = tanbeta_ds.GetRasterBand(1)
tanbetaValues = band.ReadAsArray(0, 0, cols, rows)
nodata = band.GetNoDataValue()
tanbetaValues = np.ma.masked_equal(tanbetaValues, nodata)
tanbetaValues = np.where(tanbetaValues == 0, 0.01, tanbetaValues)
#tanbetaValues = ma.masked_less(tanbetaValues, 0)

ldd_ds = gdal.Open(ldd, GA_ReadOnly)
band = ldd_ds.GetRasterBand(1)
lddValues = band.ReadAsArray(0, 0, cols, rows)
nodata = band.GetNoDataValue()
lddValues = np.ma.masked_equal(lddValues, nodata)
#lddValues = ma.masked_less(lddValues, 0)
ldd_dict = {1: [1, -1],
            2: [1, 0],
            3: [1, 1],
            4: [0, -1],
            5: [0, 0],
            6: [0, 1],
            7: [-1, -1],
            8: [-1, 0],
            9: [-1, 1]    
}
structure = [[1,1,1],
             [1,1,1],
             [1,1,1]]
    
#initialize to all cells saturated
waterStorageFile = working_dir + watershed_nick + "_storage.tif"
initial_storage = D*resol*resol*fi
waterStorageArray = np.where(elevationValues>0.0, initial_storage, None)
createRaster(waterStorageFile, waterStorageArray, coords_system, resol, raster_origin)
waterStorage_ds = gdal.Open(waterStorageFile, GA_ReadOnly)
waterStorage_geotransform = waterStorage_ds.GetGeoTransform()
print(waterStorage_geotransform[0], waterStorage_geotransform[3], resol)
band = waterStorage_ds.GetRasterBand(1)
waterSoilStorageValues = band.ReadAsArray(0, 0, cols, rows)
saturatedWatershedWaterVolume = np.sum(waterSoilStorageValues)

wTableHeightFile = working_dir + watershed_nick + "_water_table_height.tif"
wTableHeightArray = np.where(elevationValues>0.0, D, None)
createRaster(wTableHeightFile, wTableHeightArray, coords_system, resol, raster_origin)
wTableHeight_ds = gdal.Open(wTableHeightFile, GA_ReadOnly)
tableHeight_geotransform = wTableHeight_ds.GetGeoTransform()
print(tableHeight_geotransform[0], tableHeight_geotransform[3], resol)
band = wTableHeight_ds.GetRasterBand(1)
wTableHeightValues = band.ReadAsArray(0, 0, cols, rows)

# these lists contain the values to build the area ratio vs soil content curve
Area_ratio_list = []
normalized_Vsoilwater_list = []
saturFront_list = []
watershed_cells = np.where(elevationValues>0.0, 1, 0)
#createRaster(working_dir + watershed_nick + "_watershed_cells.tif", watershed_cells, coords_system, resol, raster_origin)
total_area = np.sum(watershed_cells) * resol * resol
watershed_cells = np.where(elevationValues>0.0, 1, None)
saturatedWatershedSoilStorage = np.where(watershed_cells==1, waterSoilStorageValues, 0)
saturatedWatershedSoilWaterVolume = np.sum(saturatedWatershedSoilStorage)


outlet_cell = list([-int((outlet_coords[1] - originY) / resol), int((outlet_coords[0] - originX) / resol)])
gauges_cells = []
for point in gauges_coords:
    gauges_cells.append([-int((point[1] - originY) / resol), int((point[0] - originX) / resol)])
print(gauges_coords, gauges_cells)


run_foldername = "{}_{}_{}".format(watershed_nick, n_days, run_number, m) + os.path.sep
if decaying_transmissivity == True:
    run_foldername = "{}_{}_{}_decay{}".format(watershed_nick, n_days, run_number, m) + os.path.sep
if not os.path.exists(run_foldername):
    os.mkdir(run_foldername)
print(run_foldername)


start_time = time.perf_counter()
drainTheWatershed(watershed_cells, elevationValues, tanbetaValues, lddValues, resol, structure, run_foldername,
                   originX, originY, outlet_cell, gauges_cells,
                   Area_ratio_list, normalized_Vsoilwater_list, saturFront_list,
                   waterSoilStorageValues, wTableHeightValues,
                   total_area, saturatedWatershedSoilWaterVolume,
                   D, fi, Sy, Ks, m, psi_b, n_days, deltat,
                   decaying_transmissivity)
if decaying_transmissivity == True:
    relationship_table_filename = run_foldername+watershed_nick+"_{}_{}_decay{}_ARatio_satFront_SoilWatVol.txt".format(n_days, run_number, m)
else:
    relationship_table_filename = run_foldername+watershed_nick+"_{}_{}_noDecay_ARatio_satFront_SoilWatVol.txt".format(n_days, run_number)
print_Aratio_SoilWatVol_relationship(relationship_table_filename,
                                     Area_ratio_list,
                                     normalized_Vsoilwater_list,
                                    saturFront_list)
print("\nSaturated area ratio at the end of the simulation is {}%.".format(100 * Area_ratio_list[-1]))
print("The model ran {} days in {} seconds in {} watershed.".format(n_days, 
                                                                    (time.perf_counter() - start_time),
                                                                    watershed_name))
    
plotCurve(normalized_Vsoilwater_list, Area_ratio_list, saturFront_list,
          run_foldername, watershed_name, watershed_nick,
          decaying_transmissivity, m)


#main()
#if __name__ == __main__:
#    main()

In [ ]:
################################ SANDBOX ONLY BELOW  ################################
a = np.array([[0,1,1,0],[1,1,1,1],[0,0,1,0],[1,0,1,0]])
print(a)
print(np.roll(a,(0,-1),(1,0)))

In [ ]:
east = np.roll(a,(-1,0),(1,0))
west = np.roll(a,(1,0),(1,0))
north = np.roll(a,(0,-1),(1,0))
south = np.roll(a,(0,1),(1,0))
change = 0
change += np.sum(np.where(east != a,1,0))
#change += np.sum(np.where(west != a,1,0))
change += np.sum(np.where(north != a,1,0))
#change += np.sum(np.where(south != a,1,0))
print(change)
a[1][1]

change1 = 0
change1 += np.sum(a[:,1:] != a[:,:-1]) + np.sum(a[1:,:] != a[:-1,:])
change1 += np.sum(a[0,:] == 1) + np.sum(a[-1,:] == 1)
change1 += np.sum(a[:,0] == 1) + np.sum(a[:,-1] == 1)

print(change1)

structure = [[1,1,1],
             [1,1,1],
             [1,1,1]]
contiguous_saturated = scipy.ndimage.label(b, structure)[0]

In [ ]:
b = np.array([[0,0,1,1,0],[1,1,1,1,0],[1,1,1,0,1],[1,1,0,0,0],[1,1,1,0,1]])
c = np.array([[0,0,1,1,0],[1,1,1,1,0],[0,0,1,0,0],[1,0,0,0,0],[1,1,1,0,1]])
print(b)
print()
print(c)
print('\n')
structure = [[1,1,1],
             [1,1,1],
             [1,1,1]]
contiguous_saturated = scipy.ndimage.label(c, structure)[0]
contiguous_saturated = np.where(contiguous_saturated == 1, 1, 0)
previous_contiguous_saturated = scipy.ndimage.label(b, structure)[0]
previous_contiguous_saturated = np.where(previous_contiguous_saturated == 1, 1, 0)
print(previous_contiguous_saturated)
print()
print(contiguous_saturated)
previous_perimeter = 0
previous_perimeter += np.sum(previous_contiguous_saturated[:,1:] != previous_contiguous_saturated[:,:-1])
previous_perimeter += np.sum(previous_contiguous_saturated[1:,:] != previous_contiguous_saturated[:-1,:])

contiguous_saturated, previous_contiguous_saturated
#satur_to_nonsatur = np.where(contiguous_saturated != previous_contiguous_saturated, 200, None)
satur_to_nonsatur_only = np.where(contiguous_saturated != previous_contiguous_saturated, 1, None)
print(satur_to_nonsatur_only)
satur_to_nonsatur_for_perimeter = np.where(contiguous_saturated != previous_contiguous_saturated, 200, contiguous_saturated)
satur_to_nonsatur_only_rows, satur_to_nonsatur_only_cols = satur_to_nonsatur_only.shape
print(satur_to_nonsatur_for_perimeter)
print()
current_perimeter = previous_perimeter
for i in range(satur_to_nonsatur_only_rows):
    for j in range(satur_to_nonsatur_only_cols):
        if satur_to_nonsatur_for_perimeter[i][j] != 200:
            continue

        neighborhood = []

        if j==0:
            west = None
        else:
            west = satur_to_nonsatur_for_perimeter[i][j-1]
        neighborhood.append(west)

        if i==0:
            north = None
        else:
            north = satur_to_nonsatur_for_perimeter[i-1][j]
        neighborhood.append(north)

        try:
            east = satur_to_nonsatur_for_perimeter[i][j+1]
        except:
            east = None
        neighborhood.append(east)

        try:
            south = satur_to_nonsatur_for_perimeter[i+1][j]
        except:
            south = None
        neighborhood.append(south)

        print(i, j, neighborhood)
        for neighbor in neighborhood:
            if neighbor == 0:
                current_perimeter -= 1
            if neighbor == 1:
                current_perimeter += 1
print(current_perimeter)        
